## Normalization

In [1]:
import h5py
import torch
import numpy as np
import matplotlib.pyplot as plt

# with h5py.File(f'data_large/Burgers_train_100000_default.h5', 'r') as f:
#     # Traj_dataset.traj_train = torch.tensor(f['train']['pde_140-256'][:10000, :131], dtype=torch.float32, device=cfg.device)
#     traj = torch.tensor(f['train']['pde_140-256'][:1000, :131], dtype=torch.float32)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
import hydra
from omegaconf import DictConfig, OmegaConf
from generate_data import generate_timestep
from tqdm import tqdm

hydra.initialize(config_path="cfg_time_batch", version_base=None)
cfg = hydra.compose(config_name="config", overrides=["task=KS", "nt=14"])

from utils import set_seed
set_seed(100)

In [3]:
for scale in [0.125, 0.25, 0.5, 1.0, 2.0, 4.0]:
    print(scale)

    with h5py.File(cfg.dataset.train_path, 'r') as f:
        # Traj_dataset.traj_train = torch.tensor(f['train']['pde_140-256'][:10000, :131], dtype=torch.float32, device=cfg.device)
        traj = torch.tensor(f['train']['pde_140-256'][:1000, :131], dtype=torch.float32)

    mean = traj[:32].mean()
    std = traj[:32].std()
    print(f'Mean: {mean}, Std: {std}')
    traj = (traj - mean) / std * scale

    from neuralop.models import FNO

    unrolling = cfg.train.unrolling
    nt = cfg.nt
    ensemble_size = cfg.ensemble_size
    num_acquire = cfg.num_acquire
    device = cfg.device
    epochs = cfg.train.epochs
    lr = cfg.train.lr
    batch_size = cfg.train.batch_size
    initial_datasize = cfg.initial_datasize

    def train(Y, train_nts, **kwargs):
        model = FNO(n_modes=(256, ), hidden_channels=64,
                    in_channels=1, out_channels=1)

        model = model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
        criterion = torch.nn.MSELoss()

        inputs = []
        outputs = []
        for b in range(Y.shape[0]):
            for t in range(train_nts[b].item()-1):
                inputs.append(Y[b,t])
                outputs.append(Y[b, t+1])
        inputs = torch.stack(inputs, dim=0).unsqueeze(1)
        outputs = torch.stack(outputs, dim=0).unsqueeze(1)

        dataset = torch.utils.data.TensorDataset(inputs, outputs)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

        model.train()
        for epoch in tqdm(range(epochs)):
            model.train()
            # max_unrolling = epoch if epoch <= unrolling else unrolling
            # unrolling_list = [r for r in range(max_unrolling + 1)]

            total_loss = 0
            for x, y in dataloader:
                optimizer.zero_grad()
                x, y = x.to(device), y.to(device)
                
                pred = model(x)
                loss = criterion(pred, y)

                # loss = torch.sqrt(loss)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            scheduler.step()
            # wandb.log({f'train/loss_{acquire_step}': total_loss})
        return model

    timestep = (traj.shape[1] - 1) // (nt - 1) # 10
    Y = traj[:,0::timestep]
    train_nts = torch.ones(Y.shape[0], device=device, dtype=torch.int64)
    train_nts[:initial_datasize] = nt

    model = train(Y, train_nts)

    from utils import torch_expand
    from eval_utils import compute_metrics

    test_Y = traj[initial_datasize:1000, 0::timestep]

    model.eval()

    preds = []
    preds.append(test_Y[:,0:1])

    with torch.no_grad():
        for t in range(nt-1):
            X = preds[-1].to(device)
            pred = model(X) # batch x 1 x 256
            preds.append(pred.cpu())

    preds = torch.cat(preds, dim=1) # batch x nt x 256


    metrics = compute_metrics(test_Y, preds, d=2, device=device, reduction=True)

    print(metrics)

print('max-min scaling')

with h5py.File(cfg.dataset.train_path, 'r') as f:
    # Traj_dataset.traj_train = torch.tensor(f['train']['pde_140-256'][:10000, :131], dtype=torch.float32, device=cfg.device)
    traj = torch.tensor(f['train']['pde_140-256'][:1000, :131], dtype=torch.float32)

max = traj.max()
min = traj.min()
print(f'Max: {max}, Min: {min}')
traj = (traj - (max+min/2)) / (max - min)

from neuralop.models import FNO

unrolling = cfg.train.unrolling
nt = cfg.nt
ensemble_size = cfg.ensemble_size
num_acquire = cfg.num_acquire
device = cfg.device
epochs = cfg.train.epochs
lr = cfg.train.lr
batch_size = cfg.train.batch_size
initial_datasize = cfg.initial_datasize

def train(Y, train_nts, **kwargs):
    model = FNO(n_modes=(256, ), hidden_channels=64,
                in_channels=1, out_channels=1)

    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
    criterion = torch.nn.MSELoss()

    inputs = []
    outputs = []
    for b in range(Y.shape[0]):
        for t in range(train_nts[b].item()-1):
            inputs.append(Y[b,t])
            outputs.append(Y[b, t+1])
    inputs = torch.stack(inputs, dim=0).unsqueeze(1)
    outputs = torch.stack(outputs, dim=0).unsqueeze(1)

    dataset = torch.utils.data.TensorDataset(inputs, outputs)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model.train()
    for epoch in tqdm(range(epochs)):
        model.train()
        # max_unrolling = epoch if epoch <= unrolling else unrolling
        # unrolling_list = [r for r in range(max_unrolling + 1)]

        total_loss = 0
        for x, y in dataloader:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            
            pred = model(x)
            loss = criterion(pred, y)

            # loss = torch.sqrt(loss)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        scheduler.step()
        # wandb.log({f'train/loss_{acquire_step}': total_loss})
    return model

timestep = (traj.shape[1] - 1) // (nt - 1) # 10
Y = traj[:,0::timestep]
train_nts = torch.ones(Y.shape[0], device=device, dtype=torch.int64)
train_nts[:initial_datasize] = nt

model = train(Y, train_nts)

from utils import torch_expand
from eval_utils import compute_metrics

test_Y = traj[initial_datasize:1000, 0::timestep]

model.eval()

preds = []
preds.append(test_Y[:,0:1])

with torch.no_grad():
    for t in range(nt-1):
        X = preds[-1].to(device)
        pred = model(X) # batch x 1 x 256
        preds.append(pred.cpu())

preds = torch.cat(preds, dim=1) # batch x nt x 256


metrics = compute_metrics(test_Y, preds, d=2, device=device, reduction=True)

print(metrics)

0.125
Mean: 4.994306079808553e-10, Std: 1.3284176588058472


100%|██████████| 100/100 [01:42<00:00,  1.02s/it]


(tensor(0.0709), tensor(0.1261), tensor(1.3528))
0.25
Mean: 4.994306079808553e-10, Std: 1.3284176588058472


100%|██████████| 100/100 [01:43<00:00,  1.03s/it]


(tensor(0.1365), tensor(0.1214), tensor(5.2219))
0.5
Mean: 4.994306079808553e-10, Std: 1.3284176588058472


...

(tensor(1.5058), tensor(0.1674), tensor(565.1122))
4.0
Mean: 4.994306079808553e-10, Std: 1.3284176588058472


100%|██████████| 100/100 [01:38<00:00,  1.01it/s]


(tensor(6.3953), tensor(0.3545), tensor(8328.8721))
max-min scaling
Max: 3.5027523040771484, Min: -3.430288553237915


100%|██████████| 100/100 [01:31<00:00,  1.09it/s]


(tensor(0.1184), tensor(0.0807), tensor(3.7226))
